In [ ]:
import datetime as dt
import panel as pn
import param
from viresclient import SwarmRequest

pn.extension('codeeditor')

In [2]:
VIRES = SwarmRequest()

COLLECTION_MAP = VIRES.available_collections(details=False)
MEASUREMENTS_BY_COLLECTION = {_coll_type: VIRES.available_measurements(_coll_type) for _coll_type in COLLECTION_MAP.keys()}
AUXILIARIES = VIRES.available_auxiliaries()
AUXILIARIES = [_a for _a in AUXILIARIES if _a not in ["Timestamp", "Latitude", "Longitude", "Radius", "Spacecraft"]]
MAG_MODELS = VIRES.available_models(details=False)
COLLECTIONS_TO_TYPES = VIRES._available["collections_to_keys"]

MAG_COLLECTIONS = {
    "Swarm": {
        "Swarm-A": {"OPER LR (1Hz)": "SW_OPER_MAGA_LR_1B", "OPER HR (50Hz)": "SW_OPER_MAGA_HR_1B", "FAST LR (1Hz)": "SW_FAST_MAGA_LR_1B", "FAST HR (50Hz)": "SW_FAST_MAGA_HR_1B"},
        "Swarm-B": {"OPER LR (1Hz)": "SW_OPER_MAGB_LR_1B", "OPER HR (50Hz)": "SW_OPER_MAGB_HR_1B", "FAST LR (1Hz)": "SW_FAST_MAGB_LR_1B", "FAST HR (50Hz)": "SW_FAST_MAGB_HR_1B"},
        "Swarm-C": {"OPER LR (1Hz)": "SW_OPER_MAGC_LR_1B", "OPER HR (50Hz)": "SW_OPER_MAGC_HR_1B", "FAST LR (1Hz)": "SW_FAST_MAGC_LR_1B", "FAST HR (50Hz)": "SW_FAST_MAGC_HR_1B"},
    },
    "Cryosat-2": {
        "Cryosat-2": {"OPER": "CS_OPER_MAG"},
    },
    "GRACE": {
        "GRACE-A": {"OPER": "GRACE_A_MAG"},
        "GRACE-B": {"OPER": "GRACE_B_MAG"},
    },
    "GRACE-FO": {
        "GRACE-FO-1": {"OPER": "GF1_OPER_FGM_ACAL_CORR"},
        "GRACE-FO-2": {"OPER": "GF2_OPER_FGM_ACAL_CORR"},
    },
    "GOCE": {
        "GOCE": {"OPER": "GO_MAG_ACAL_CORR", "ML": "GO_MAG_ACAL_CORR_ML"},
    },
}

In [3]:
REQUEST_TEMPLATE = """import datetime as dt
from viresclient import SwarmRequest

request = SwarmRequest()
request.set_collection('{collection}', verbose=False)
request.set_products(
    measurements={measurements},
    models=['{magnetic_model}'],
    auxiliaries={auxiliaries},
    # sampling_step="PT1S"
)
data = request.get_between(
    start_time={time_range[0]!r},
    end_time={time_range[1]!r},
    asynchronous=False,
    show_progress=False,
)
ds = data.as_xarray()"""

In [ ]:
class ViresParameters(param.Parameterized):
    collection_type = param.Selector(default="MAG", objects=list(COLLECTION_MAP.keys()))
    collection = param.Selector(objects=COLLECTION_MAP["MAG"])
    measurements = param.ListSelector(default=[], objects=MEASUREMENTS_BY_COLLECTION["MAG"])
    magnetic_model = param.Selector(default="", objects=[""] + MAG_MODELS)
    auxiliaries = param.ListSelector(default=[], objects=AUXILIARIES)
    time_range = param.DateRange(default=(dt.datetime(2024, 3, 1), dt.datetime(2024, 3, 1, 0, 1)))
    code_snippet = param.String("")
    preview_dataset_html = param.String("")

    # Magnetic (space) interface
    mission = param.Selector(default="Swarm", objects=list(MAG_COLLECTIONS.keys()))
    spacecraft = param.Selector()
    variant = param.Selector()
    mag_collection = param.String()

    @param.depends("collection_type", watch=True)
    def _update_collections_and_measurements(self):
        self.measurements = []
        collections = COLLECTION_MAP[self.collection_type]
        self.param["collection"].objects = collections
        self.collection = collections[0]
        measurements = MEASUREMENTS_BY_COLLECTION[self.collection_type]
        self.param["measurements"].objects = measurements

    @param.depends("collection", "measurements", "auxiliaries", "time_range", "magnetic_model", watch=True, on_init=True)
    def _update_code_snippet(self):
        self.code_snippet = REQUEST_TEMPLATE.format(
            collection=self.collection,
            measurements=self.measurements,
            auxiliaries=self.auxiliaries,
            time_range=self.time_range,
            magnetic_model=self.magnetic_model,
        ).replace("datetime.datetime", "dt.datetime")

    @param.depends("code_snippet", watch=True, on_init=True)
    def _update_preview_dataset(self):
        namespace = {}
        try:
            exec(self.code_snippet, namespace)
            ds = namespace["ds"]
        except Exception:
            self.preview_dataset_html = "Invalid request"
        else:
            self.preview_dataset_html = ds._repr_html_()

    @param.depends("mission", watch=True, on_init=True)
    def _update_spacecraft(self):
        spacecrafts = list(MAG_COLLECTIONS[self.mission].keys())
        self.param["spacecraft"].objects = spacecrafts
        self.spacecraft = spacecrafts[0]

    @param.depends("spacecraft", watch=True, on_init=True)
    def _update_variants(self):
        variants = list(MAG_COLLECTIONS[self.mission][self.spacecraft].keys())
        self.param["variant"].objects = variants
        self.variant = variants[0]

    @param.depends("spacecraft", "variant", watch=True, on_init=True)
    def _update_mag_collection(self):
        self.mag_collection = MAG_COLLECTIONS[self.mission][self.spacecraft][self.variant]

    @param.depends("mission", "spacecraft", "variant", "mag_collection", watch=True, on_init=True)
    def _update_collections(self):
        self.collection_type = COLLECTIONS_TO_TYPES[self.mag_collection]
        self.collection = self.mag_collection


parameter_state = ViresParameters()
html_pane = pn.pane.HTML(parameter_state.preview_dataset_html)

# Callback function to update the HTML content
def _update_html(event):
    html_pane.object = parameter_state.preview_dataset_html

# Attach the callback function to the widgets' events
parameter_state.param.watch(_update_html, "code_snippet")

dashboard = pn.Row(
    pn.layout.Tabs(
        pn.Param(
            parameter_state,
            parameters=["collection_type", "collection"],
            widgets={
                "collection": {"type": pn.widgets.Select, "size": 6},
            },
            name="Generic"
        ),
        pn.Param(
            parameter_state,
            parameters=["mission", "spacecraft", "variant", "mag_collection"],
            name="Magnetic (space)"
        ),
    ),
    pn.Param(
        parameter_state,
        parameters=["measurements"],
        widgets={
            "measurements": {"type": pn.widgets.CheckBoxGroup,}
        },
        name="Measurements",
    ),
    pn.Param(
        parameter_state,
        parameters=["magnetic_model", "auxiliaries"],
        widgets={
            "auxiliaries": {"type": pn.widgets.CheckBoxGroup,},
        },
        name="Auxiliaries"
    ),
    pn.Param(
        parameter_state,
        parameters=["time_range", "code_snippet"],
        widgets={
            "time_range": {"type": pn.widgets.DatetimeRangePicker},
            "code_snippet": {"type": pn.widgets.CodeEditor, "width": 500, "language": "python", "readonly": True, "print_margin": False}
        },
        name=""
    ),
    html_pane
)

dashboard.servable()